# Introduction to Prophet
<img src=images/gdd-logo.png align='right' width=200>

## Goal

The goal of this notebook is to introduce Prophet from Facebook. It is a procedure for forecasting time series data that works by explicitly modeling trends, seasonalities and holiday effects, making it especially suitable for business applications.

## Program

- [About Prophet](#ap)
- [Forecasting with Prophet](#fwp)
    - [Data preperation for Prophet]()
    - [Instantiating the model]()
    - [Fitting the model]()
    - [Fetting predictions from the model]()
- [Plotting with Prophet](#pwp)
    - [Plotting the forecast]()
    - [Plotting the components]()
    - [Assignment](#as1)
- [Change Points](#cp)
- [Model evaluation with Prophet](#mewp)

- [Multiplicative Seasonality and Model Selection](#msms)
- [Handling special events]()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from prophet import Prophet

%matplotlib inline
plt.rcParams["font.size"] = 14
plt.rcParams['figure.figsize'] = (20.0, 10.0)

<a id='ap'></a>
## About Prophet

<!-- [Installation guide for Prophet](https://facebook.github.io/prophet/docs/installation.html#python)

You may also want to install `plotly` (4.0 or above) and `ipywidgets` for interactive plots. -->

<a href="https://facebookincubator.github.io/prophet/">
<img src='images/prophet.png' align='center' style="width:300px"></a> 

Prophet is a libary for forecasting time series data. It was released by Facebook’s Core Data Science team on [GitHub](https://github.com/facebookincubator/prophet) in February 2017. Models are "fit" with [Stan](http://mc-stan.org/) and you can read the paper [here](https://facebookincubator.github.io/prophet/static/prophet_paper_20170113.pdf).

### How Prophet works

Prophet is an additive regression model, which models Time Series data with three main components: trend $T$, seasonality $S$ and holidays $H$

$$Y(t) = T(t) + S(t) + H(t) + e_t$$

**Trend**

By default, a piecewise linear model is used to capture the trend.

A piecewise linear model joins together individual linear models at *breakpoints* where the trend chagnes.

These breakpoints can be automatically identified by the model or set manually.

Alternatively, a logistic model can be used for non-linear growth with saturation or a flat model if there is no upwards/downards trend.

**Seasonality**

Fourier series are used to model seasonality.

Dummy variables can be introduced when the seasonality is not constant throughout a period. For example, a weekly seasonal pattern that is different during the summer than it is during the rest of the year, or a daily seasonal pattern that is different on weekends vs. on weekdays.

**Holidays**

Users can provide a list of repeating/non-repeating special events, such as holidays, to prevent these irregularities being captured by the trend component.

**Residual error**

The residual represents any idiosyncratic changes which are not accommodated by the model

<a id='fwp'></a>
## Forecasting with Prophet

To demonstrate Prophet, we shall use a dataset that contains the total monthly passenger traffic at [Schiphol airport](https://www.schiphol.nl/en/schiphol-group/page/transport-and-traffic-statistics/) between 1999 and 2019. Each datapoint indicates the total number of passengers traveled via Schiphol in millions in each particular month.

In [ ]:
schiphol = pd.read_csv('data/schiphol_passengers.csv', 
                       index_col='date', 
                       parse_dates=True,
                       #date_format="%Y %B" for Pandas 2.0+
                      )
schiphol.index.freq = 'MS'
schiphol.head()

![](images/schiphol.png)

### Data preparation for Prophet

Prophet expects a `ds` column with timestamps and he variable of interest should be called `y`.

In [ ]:
schiphol = (schiphol
            .rename(columns={'total_passengers': 'y'})
            .assign(y=lambda df: df['y']/1000000,  #the variable of interest should be called 'y' for Prophet
                   ds=lambda df: df.index)  #Prophet needs a 'ds' column with timestamps
           )
schiphol.tail()

We shall create a train and test split so we can evaluate the generalisability of our models.

In [ ]:
schiphol_train = schiphol.loc[:'2017 Jan']
schiphol_test = schiphol.loc['2017 Jan':]
print(f"{len(schiphol_train)} months of training data")
print(f"{len(schiphol_test)} months of test data")

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
schiphol_train['y'].plot(ax=ax);
schiphol_test['y'].plot(ax=ax, c='g');

### Instantiating the model

The first step is to instantiate a new `Prophet` object.  

In [ ]:
model = Prophet()

This is where we determine settings such as the type of trend growth or seasonality.

In [ ]:
## By default, Prophet will fit weekly and yearly seasonalities.
#model = Prophet(yearly_seasonality=True, daily_seasonality=False)

### Fitting the model

Once the model has been initiated you can call its `fit` method and pass in the historical dataframe.

In [ ]:
model.fit(schiphol_train);

### Getting predictions from the model

Predictions are then made on a dataframe with a column `ds` containing the dates for which a prediction is to be made. You can get a suitable dataframe that extends into the future a specified number of days using the helper method `.make_future_dataframe()`. By default it will also include the dates from the history, so we will see the model fit as well.

Prophet expects daily data by default - remember to specify if you have a different format

In [ ]:
future = model.make_future_dataframe(periods=len(schiphol_test)-1, freq='MS') 
future.tail()

The `predict` method will assign each row in `future` a predicted value which it names `yhat`.  If you pass in historical dates, it will provide an in-sample fit. 

The `forecast` object here is a new dataframe that includes a column `yhat` with the forecast, as well as columns for components and uncertainty intervals. 

In [ ]:
forecast = model.predict(future)
print(forecast.shape)
forecast.head()

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Prophet estimates uncertainty by using Monte Carlo simulation. By default, 1000 samples are generated and Prophet returns the $10^{th}$ and $90^{th}$ percentiles at each time step (uncertainty interval of 80%). However, these are model parameters that can be set, e.g. `Prophet(uncertainty_samples=100, interval_width=0.95)`.

This [blogpost](https://towardsdatascience.com/how-to-run-facebook-prophet-predict-x100-faster-cce0282ca77d) contains a good explanation for how Prophet performs Monte Carlo simulation to generate uncertainty intervals.

<a id='pwp'></a>
## Plotting with Prophet

### Plotting the forecast

Prophet has its own plotting functionality, which is compatible with Matplotlib. You can plot the forecast by calling the `.plot()` method and passing in your forecast dataframe.

Black dots indicate the real data points. Dark blue line shows the model fit with light blue area indicating lower to upper prediction bounds.

In [ ]:
model.plot(forecast, figsize=(18,7));
plt.xticks(rotation=25);

An interactive figure of the forecast and components can be created with plotly. You will need to install plotly 4.0 or above separately, as it will not by default be installed with prophet. You will also need the `ipywidgets` package.

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(model, forecast)

Let's have a closer look at the prediction area and compare it to the actual  test set values:

In [ ]:
fig, ax = plt.subplots()
model.plot(forecast, ax=ax);
schiphol_test['y'].plot(ax=ax, c='green')
plt.xticks(rotation=25)
plt.xlim(pd.Timestamp('2017-01-01'), pd.Timestamp('2019-12-01')); # to zoom into a region

Predictions look quite good, especially for fall and winter months, with a bit of underestimation for the summer peaks.


### Plotting the components

If we want a more customized plot, we can simply get the necessary columns from the *forecast* dataframe and plot them as we usually would do with Pandas:

In [ ]:
forecast.shape

In [ ]:
schiphol.index.shape

In [ ]:
pltr = (forecast
         .set_index(schiphol.index)
        )

fig, ax = plt.subplots(figsize=(18,6))
schiphol['y'].plot(ax=ax);
pltr[['trend','yearly']].plot(ax=ax);
ax.legend(["y_real", "trend", "yearly seasonality"], prop={'size': 16});

<a id='as1'></a>
### Assignment: Detecting Outliers

Identify and highlight on the plot the *training set* points that lie outside of the confidence boundaries of our Prophet model

*HINT:* Create a new variable that has the real *y values* when it is an outlier and a missing value (np.nan) otherwise. Then plot it alongside Prophet's plot. Make sure the index of the new dataframe is the timestamps.

In [ ]:
# %load answers/prophet_outliers.py

Prophet has functions to automate the plotting of the componenents. 

However, with this dataset it will give strange results for the yearly seasonality plot. This is because the model was trained on monthly data but we are asking it to plot daily information. See [here](https://facebook.github.io/prophet/docs/non-daily_data.html#monthly-data) for more information.

In [ ]:
model.plot_components(forecast)
plt.xticks(rotation=25);

In [ ]:
plot_components_plotly(model, forecast)

<a id='cp'></a>

## Change Points

Often it is valuable to identify points in time at which the growth rate over time changes and breaks in the trend happen. Prophet has some useful tools to assist us with this.

Let's first have a look at our dataset and the trends in it again:

In [ ]:
schiphol['y'].plot(figsize=(18,6));

How many change points do you think there are in the data? Which are easier to identify?

Now let's consult Prophet:

In [ ]:
from prophet.plot import add_changepoints_to_plot

fig = model.plot(forecast)
c_points = add_changepoints_to_plot(fig.gca(),model, forecast)

Change point detection with Prophet is not only simple, but would typically outperform many good custom models. It can serve as a handy analytical tool to identify and better understand key periods of the time series.

If preferred, it is possible to [set the change points manually](https://facebook.github.io/prophet/docs/trend_changepoints.html#specifying-the-locations-of-the-changepoints).

<a id='mewp'></a>
## Model Evaluation with Prophet

Since Prophet provides the $\hat{y}$ column, we can compare it with the actual test data and calculate the usual metrics such as MAE or RMSE, as we would normally do with our custom models

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

y_pred = forecast.iloc[-36:]['yhat']
y_test = schiphol_test['y']
y_test_mean = round(y_test.mean(),2)

MAE = round(mean_absolute_error(y_test, y_pred),2)
MSE = round(mean_squared_error(y_test, y_pred),2)
RMSE = round(np.sqrt(MSE),3)
MAPE = round(mean_absolute_percentage_error(y_test, y_pred)*100,2)

print(f'MAE: {MAE}, RMSE: {RMSE}, MAPE: {MAPE}%')

Luckily for us, Prophet comes with its own model diagnostics functionality. It can actually do much more than just MSE on the test set. Most notably, it allows model evaluation with cross-validation - evaluate the model on several sections of our time series. This can give an even better idea of the model's predictive power.

<img src='images/prophet_cv.png' style="width:500px"></a> 

Various cutoff points are selected in the history, and for each of them the model is fitted using data only up to that cutoff point. The forecasted values are then compared to the actual values. 

In [ ]:
from prophet.diagnostics import cross_validation

First we need to set some parameters for `cross_validation`:

As we are working with monthly data it is better to specify the cutoff periods we will use. 

In [ ]:
schiphol_train['ds'].min(), schiphol_train['ds'].max()

In [ ]:
# Cutoffs - we will train on data up until the cutoff points
# i.e. 5 year inital period, then every 2 years
cutoffs = pd.date_range(start='2004-01-01', end='2016-01-01', freq='2Y')
cutoffs

However, it would also be possible to specify an initial training period and the subsequent spacing between cutoff dates.

In [ ]:
# #Initial - the size of the initial training period (3 * horizon by default)
# initial = str(5*365)+" days"
# initial

In [ ]:
# #Period - the spacing between cutoff dates - every X days  (0.5 * horizon by default)
# period = str(2*365)+" days"
# period

The horizon specifies how far into the future we want to predict.

In [ ]:
#Horizon - how far out to forecast for each period
horizon = str(365)+" days"
horizon

In [ ]:
#performing cross validation

# Using cutoffs
df_cv = cross_validation(model, cutoffs=cutoffs, horizon=horizon)

# Using inital & period
# df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)

Let's have a look at the results of cross validation:

In [ ]:
df_cv

In [ ]:
df_cv.resample('Y', on='ds').count()

Metrics for the cross-validation predictions are computed for different horizon windows:

In [ ]:
from prophet.diagnostics import performance_metrics
performance_metrics(df_cv, rolling_window = 0.1)

The size of the windows (number of simulated forecast points) is determined by the `rolling_window` argument.

- The default of `rolling_window=0.1` will incorporate 10% more predictions in each window. 
- `rolling_window=1` will compute the metric across all simulated forecast points. 

To comprehend the cross validation results it can be helpful to make a plot.

In [ ]:
from prophet.plot import plot_cross_validation_metric
plot_cross_validation_metric(df_cv, metric='rmse');

In [ ]:
#mape - mean absolute percentage error
plot_cross_validation_metric(df_cv, metric='mape');

<a id='msms'></a>
## Multiplicative Seasonality and Model Selection

While it may feel like Prophet does all the work for you, there are some important parameters that you can change to create a better model. For instance, you can manually set the types of seasonality that you expect in the data (even though Prophet can also do this automatically). You can also choose to not have constant seasonality over time - by default Prophet will assume fixed seasonality.

Let's inspect our Prophet model once again and pay a closer attention to the seasonal cycles:

In [ ]:
model.plot(forecast, figsize=(18,7));
plt.xticks(rotation=25);
plt.xlim(pd.Timestamp('1999-01-01'), pd.Timestamp('2017-01-01'));

We can notice that our seasonal cycles are a bit too large for the beginning and a bit too small for the latest data points. This could be a likely sign of *Multiplicative Seasonality*. By setting `seasonality_mode='multiplicative'` we can allow Prophet to fit a model with growing (or shrinking) seasonal cycles over time.

In [ ]:
model2 = Prophet(seasonality_mode='multiplicative')
model2.fit(schiphol_train);


future2 = model2.make_future_dataframe(periods=len(schiphol_test)-1, freq='MS') 
forecast2 = model2.predict(future2)

fig, ax = plt.subplots()
model2.plot(forecast2, ax=ax);
plt.xticks(rotation=25);

This seems to be a much better fit! The seasonal cycles are just large enough for all moments in time. Also notice how the confidence bounds are narrower - the model is more confident about its fit and predictions now.

Ironically, a better fit does not always mean better predictions. This new model is quite confident that summer passenger traffic will be expanding more and more after 2017. This is not unreasonable given the past data. However the reality (airport capacity and environmental challenges) make the reality a bit different: 

In [ ]:
fig, ax = plt.subplots()
model2.plot(forecast2, ax=ax);
schiphol_test['y'].plot(ax=ax, c='r');
plt.xticks(rotation=25);

We actually make more errors with this new model, when making predictions into 2017-2019, even though it fits the historic data better:

In [ ]:
y_pred2 = forecast2.iloc[-36:]['yhat']

MAE2 = round(mean_absolute_error(y_test, y_pred2),2)
RMSE2 = round(mean_squared_error(y_test, y_pred2, squared=False),2)
MAPE2 = round(mean_absolute_percentage_error(y_test, y_pred)*100,2)

print(f'RMSE: {RMSE2}')
print(f'Whilst previously we had RMSE: {RMSE}')

Note how this is not clear from cross-validated scores - they are primarily based on the historic data where *Model 2* usually does better. These scores therefore may not properly alarm us that the assumption of further expanding seasonality of *Model 2* may hinder our predictions post-2017:

In [ ]:
#performing cross validation with the new model
df_cv2 = cross_validation(model2, cutoffs=cutoffs, horizon=horizon)
# df_cv2 = cross_validation(model2, initial=initial, period=period, horizon=horizon)

In [ ]:
fig, ax = plt.subplots()
plot_cross_validation_metric(df_cv, metric='rmse', ax=ax)
plot_cross_validation_metric(df_cv2, metric='rmse', ax=ax);
plt.gca().get_lines()[3].set_color("green")
ax.legend(["", "model 1", "", "model 2"], prop={'size': 18});

## Handling special events

Prophet has great functionaliy for handling special events and dramatic shocks in the data.

To demonstrate this, let's load in the total monthly passenger traffic at [Schiphol airport](https://www.schiphol.nl/en/schiphol-group/page/transport-and-traffic-statistics/) between January 1999 and March 2023. This will include the months passenger traffic was affected by the COVID-19 pandemic.

In [ ]:
schiphol_covid = (
    pd.read_csv('data/schiphol_covid.csv', 
        index_col='date', 
        parse_dates=True,
        #date_format="%Y %B" for Pandas 2.0+
        )
        .rename(columns={'total_passengers': 'y'})
        .assign(y=lambda df: df['y']/1000000,  #the variable of interest should be called 'y' for Prophet
                ds=lambda df: df.index)  #Prophet needs a 'ds' column with timestamps
)
schiphol_covid.index.freq = 'MS'

schiphol_covid_train = schiphol_covid.loc[:'2022 Jan']
schiphol_covid_test = schiphol_covid.loc['2022 Jan':]
print(f"{len(schiphol_covid_train)} months of training data")
print(f"{len(schiphol_covid_test)} months of test data")

fig, ax = plt.subplots(figsize=(18,5))
schiphol_covid_train['y'].plot(ax=ax);
schiphol_covid_test['y'].plot(ax=ax, c='g');

The model we created prevously will struggle to handle this "shock".

In [ ]:
%%capture 
model_covid = Prophet()

model_covid.fit(schiphol_covid_train)

future_covid = model_covid.make_future_dataframe(periods=len(schiphol_covid_test)-1, freq='MS')

forecast_covid = model_covid.predict(future_covid)

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
model_covid.plot(forecast_covid, ax=ax);
plt.xticks(rotation=25);

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
schiphol_covid['y'].plot(ax=ax);
forecast_covid.set_index("ds").plot(y="yhat", ax=ax);
ax.legend(["y_real", "y_hat"], prop={'size': 16});
plt.xticks(rotation=25);

However, if we provide prophet with the dates when this irregularity occured, we can avoid it being captured in the trend component.

We need to create a DataFrame with two columns:
- `holiday`: name for the event
- `ds`: date of the event

We shall estimate when COVID-19 brought lockdowns to the Netherlands.

In [ ]:
covid_dates = pd.date_range(start='2020 April', end='2022 March')

In [ ]:
lockdowns = pd.DataFrame(
    columns=['holiday', 'ds'],
    data=[["day "+str(idx), date] for idx, date in enumerate(covid_dates)]
)
lockdowns

Alternatively, we can provide an upper and lower bound if it is a repeated event.

In [ ]:
lockdowns = pd.DataFrame([{
    'holiday': 'lockdown', 'ds': pd.Timestamp('2020-04-01'), 'lower_window': 0, 'upper_window':len(covid_dates)-1
}])
lockdowns

We can then pass this information into the Prophet object

In [ ]:
%%capture 
model_lockdowns = Prophet(holidays=lockdowns)

model_lockdowns.fit(schiphol_covid_train)

future_lockdowns = model_lockdowns.make_future_dataframe(periods=len(schiphol_covid_test)-1, freq='MS')

forecast_lockdowns = model_lockdowns.predict(future_lockdowns)

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
model_lockdowns.plot(forecast_lockdowns, ax=ax);
plt.xticks(rotation=25);

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
schiphol_covid['y'].plot(ax=ax);
forecast_lockdowns.set_index("ds").plot(y="yhat", ax=ax);
ax.legend(["y_real", "y_hat"], prop={'size': 16});
plt.xticks(rotation=25); 

Whilst we still do not have a perfect model, it is clearly a better estimation than before.

<a id='sum'></a>
## Summary

This notebook has provided you with an overview on how to use the Prophet library to model Time Series data. You can refer to the [docs](https://facebook.github.io/prophet/docs/quick_start.html#python-api) for more information and further features.

**Pros:** 
- Highly automated and simple to use. 
- Does not require extensive preprocessing and domain knowledge. 
- Robust to missing data, outliers and dramatic changes. 
- Measures explainable and quantifiable effects. 
- Can be easily used to detect outliers and trend change points. 
- Typically outperforms many custom models.

**Cons:** 
- Is a bit of a black box. 
- Still in development and some features are still missing ([updates for 2023](https://medium.com/@cuongduong_35162/facebook-prophet-in-2023-and-beyond-c5086151c138)). 
- Less customization of the model.